In [20]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import json
import scipy.io as sio

In [21]:

output_path = "./submit/best_submission.json"
l1 = glob.glob("./output/type_0_0/preds/*.npy")
l2 = glob.glob("./output/type_0_1/preds/*.npy")
l3 = glob.glob("./output/type_1_0/preds/*.npy")
l4 = glob.glob("./output/type_1_1/preds/*.npy")

l = l1 + l2 + l3 + l4

In [22]:
def process_predictions(file_paths, aug=1, fold5=1):
    pred_all = []
    for file_path in file_paths:
        tmp_pred = np.load(file_path)
        pred_all.append(tmp_pred)
    pred_all = np.concatenate(pred_all, axis=2)
    
    print(pred_all.shape)

    if fold5 == 1:
        pred_all = np.median(pred_all, axis=2)
        #pred_all = np.mean(pred_all, axis=2)
    else:
        pred_all = pred_all[:, :, 0]

    pred_all = pred_all.reshape(-1, 90)
    if aug == 1:
        pred_all_even = pred_all[::2]
        pred_all_odd = pred_all[1::2]
        pred_all_result = (pred_all_even + pred_all_odd) / 2
    else:
        pred_all_result = pred_all

    pred_all_result = pred_all_result.reshape(-1, 3, 30)
    pred_all_result[:, 0, :] = -pred_all_result[:, 0, :]
    pred_all_result[:, 1, :] = -pred_all_result[:, 1, :]
    pred_all = pred_all_result
    pred = pred_all[-320:]
    return pred, pred_all


In [23]:
_, pred = process_predictions(l, aug=1, fold5=1)

(3158, 90, 20)


In [24]:
pred.shape

(1579, 3, 30)

In [26]:
# 提出用のファイルを作成
with open("../input/sample_submit.json") as r:
    sample_submit = json.load(r)

keys_1 = sample_submit.keys()
user_trial_counts = [319,300,320,320]

ind = 0
for key,user_trial_count in zip(keys_1, user_trial_counts):
    for i, trial_key in enumerate(sample_submit[key].keys()):
        sample_submit[key][trial_key] = pred[i+ind].T
    ind += user_trial_count

# 予測結果の中身に問題がないことの確認
for k in sample_submit.keys():
    for kk in sample_submit[k].keys():
        # (30, 3)の形でなければ例外を出す
        if np.array(sample_submit[k][kk]).shape != (30, 3):
            print(f"invalid: {k} {kk}")

def ndarray_to_list(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")
with open(output_path, "w") as f:
    json.dump(sample_submit, f, default=ndarray_to_list)